# Imports + Colab

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/ProgettoNAPDE/ProgettoNAPDE

ModuleNotFoundError: No module named 'google'

In [ ]:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

--2024-10-18 05:44:10--  https://fem-on-colab.github.io/releases/firedrake-install-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4581 (4.5K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.47K  --.-KB/s    in 0s      

2024-10-18 05:44:11 (72.7 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4581/4581]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/13b09a2/rele

/usr/local/lib/python3.10/dist-packages/pytools/persistent_dict.py:63: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


In [ ]:
%cd ../../../PyGeM
%ls

/gdrive/My Drive/PyGeM
build/             dist/         LICENSE.rst      readme/    test.py
CITATION.cff       dockerfiles/  pygem/           README.md  tests/
code_formatter.sh  docs/         pygem.egg-info/  setup.py   tutorials/


In [ ]:
!python3.10 setup.py install
import pygem

In [ ]:
%cd ../Colab Notebooks/ProgettoNAPDE/ProgettoNAPDE
%ls

## Imports

In [ ]:
from firedrake import *
import matplotlib.pyplot as plt
from pygem import FFD, RBF
import numpy as np

# Function definitions

## Stokes Problem 

In [ ]:
def Stokes(M):

    # function space
    V=VectorFunctionSpace(M, 'P', 2) # velocity
    Q=FunctionSpace(M, 'P', 1) # pressure
    W=MixedFunctionSpace([V, Q])

    # trial and test functions
    u, p=TrialFunctions(W)
    v, q=TestFunctions(W)

    # data
    x=SpatialCoordinate(M)
    Uinf=as_vector([1,0])

    # problem
    a=inner(grad(u), grad(v))*dx - p*div(v)*dx+ q*div(u)*dx
    L=inner(Constant((0,0)),v)*dx

    # Dirichlet BC
    bc1=DirichletBC(W.sub(0), as_vector([0,0]), 0)
    bc2=DirichletBC(W.sub(0), Uinf, 1)
    bcs=(bc1, bc2)

    #solution
    w_stokes=Function(W)
    solve(a==L, w_stokes, bcs=bcs)
    # u_h, p_h=split(wh) no split command works only for trial and test funtion
    u_stokes, p_stokes=w_stokes.subfunctions

    return w_stokes

## Navier Stokes with SUPG

In [ ]:
# Adimensionalized formulation for high Re
def functional_high_Re(uh, v, ph, q, Re, f):
    G = inner(dot(grad(uh), uh), v) * dx  \
        +  1/Re * inner(grad(uh), grad(v)) * dx  \
        - div(v) * ph * dx  \
        + q * div(uh) * dx \
        - inner(f, v) * dx
    return G

def NavierStokesSUPG(M,Re,w_stokes):

    def a(u,v, Re):
        return 1/Re*inner(grad(u), grad(v))*dx

    def c(w,u,v):
        return inner(dot(grad(u), w),v)*dx

    def b(v,q):
        return -q*div(v)*dx

    def stabilization(u_old, u, p, v, q, M, Re):
        ubar = Function(FunctionSpace(M, 'DG', 0))
        ubar.project(sqrt(inner(u_old, u_old)))
        h = CellDiameter(M)
        Re_K = h * ubar * Re
        one  = Constant(1.0)
        delta=Constant(1.0)
        delta_K = delta * conditional(gt(Re_K, one),  h/(ubar+1e-5), h*h*Re)

        L= -1.0/Re*div(grad(u))+ dot(grad(u_old), u) + dot(grad(u), u_old)+ grad(p)
        Lss = dot(grad(u_old),v) + dot(grad(v),u_old) + grad(q)

        lhs= delta_K * inner(L, Lss)*dx + delta_K*div(u)*div(v)*dx
        rhs=delta_K*inner(dot(grad(u_old), u_old), Lss)*dx

        return lhs, rhs

    # Function spaces (mixed formulation)
    V = VectorFunctionSpace(M, 'P', 2)
    Q = FunctionSpace(M, 'P', 1)
    W = MixedFunctionSpace([V, Q])

    # Data and boundary conditions
    f = Constant((0.,0.))

    u_in = as_vector([1., 0.])

    bc1 = DirichletBC(W.sub(0), Constant((0.,0.)), 0) # Dirichlet no-slip B.C. on the airfoil
    bc2 = DirichletBC(W.sub(0), u_in, 1) # Dirichlet unitary B.C. on the inflow boundary

    bcs = (bc1, bc2)
    # Trial and test functions
    u, p = TrialFunctions(W) # trial functions
    v, q = TestFunctions(W) # test functions

    param = {'ksp_type':'gmres',
        'ksp_pc_type':'ilu',
        'ksp_maxit':1000,
        'ksp_rtol':1e-8,
        'snes_rtol':1e-3,
        'snes_maxit':100,
    }

    # solution
    wh = Function(W)
    uh, ph = wh.subfunctions
    wh.assign(w_stokes) #initialization with stokes solution

    maxit=100
    tol=1e-8
    it=0
    err=tol+1
    delta=1

    u_old = Function(V)
    u_old.assign(uh)
    p_old = Function(Q)
    p_old.assign(ph)

    while it < maxit and err > tol:
        lhs, rhs = stabilization(u_old, u, p, v, q, M, Re)
        G = a(u,v,Re) + c(u,u_old,v)+c(u_old,u,v)-b(u,q)+b(v,p)+lhs
        L = inner(f,v)*dx +rhs + c(u_old, u_old,v)
        pb = LinearVariationalProblem(G, L , wh, bcs=bcs)
        solver = LinearVariationalSolver(pb, solver_parameters=param)
        solver.solve()
        uh, ph = wh.subfunctions
        err = (errornorm(uh, u_old, 'H1') / norm(u_old, 'H1') + errornorm(ph, p_old, 'L2') / norm(p_old, 'L2'))
        u_old.assign(uh)
        p_old.assign(ph)
        it+=1
        uh, ph = wh.subfunctions
        outfileU = File("output/velocity.pvd")
        outfileP = File("output/pressure.pvd")
        uh.rename("Velocity")   # this name will be used in Paraview
        ph.rename("Pressure")   # this name will be used in Paraview
        outfileU.write(uh)
        outfileP.write(ph)

    return wh
